# Trend based strategy

In [1]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr
import xarray.ufuncs as xruf

import numpy as np
import pandas as pd

# Import quantiacs libraries.

import qnt.data as qndata          # data loading and manipulation
import qnt.stats as qnstats        # key statistics
import qnt.graph as qngraph        # graphical tools
import qnt.forward_looking as qnfl # forward looking checking
import qnt.xr_talib as xrtl        # technical analysis indicators

# display function for fancy displaying:
from IPython.display import display
# lib for charts
import plotly.graph_objs as go
import datetime as dt

In [2]:
# Load all available data since given date.

# It is possible to set a max_date in the call in order to
# develop the system on a limited in-sample period and later
# test the system on unseen data after max_date.

# A submission will be accepted only if no max_date is set,
# as submissions will be evaluated on live data on a daily basis.

data = qndata.load_data(tail=dt.timedelta(days=4*365),
                        dims=("time", "field", "asset"),
                        forward_order=True
                       )

fetched chunk 1/5 1s
fetched chunk 2/5 3s
fetched chunk 3/5 4s
fetched chunk 4/5 6s
fetched chunk 5/5 7s
Data loaded 7s


We will use WMA and ROCP from qnt.xr_talib to measure trend.

In [3]:
help(xrtl.WMA)

Help on function WMA in module qnt.xr_talib:

WMA(data: xarray.core.dataarray.DataArray, timeperiod: int = 30) -> xarray.core.dataarray.DataArray
    Weighted Moving Average (Overlap Studies)
    Parameters:
        timeperiod: 30
    Input:
        data: time series
    Output:
        double series



In [4]:
help(xrtl.ROCP)

Help on function ROCP in module qnt.xr_talib:

ROCP(data: xarray.core.dataarray.DataArray, timeperiod: int = 14) -> xarray.core.dataarray.DataArray
     Rate of change Percentage: (real-prevPrice)/prevPrice (Momentum Indicators)
    Input:
        data: time series
    Parameters:
        timeperiod: 14
    Output:
        double series



Let's implement strategy based on WMA using one asset:

In [5]:
stock_name = 'NASDAQ:AAPL'

# select only 1 stock
stock = data.sel(asset=stock_name).dropna('time', 'all')

pd_time = stock.time.to_pandas()
close = stock.sel(field='close')
is_liquid = stock.sel(field='is_liquid') > 0

# chart with prices
price_fig = [
   go.Candlestick(
       x=stock.time.to_pandas(),
       open=stock.sel(field='open').values,
       high=stock.sel(field='high').values,
       low=stock.sel(field='low').values,
       close=stock.sel(field='close').values,
       name=stock_name
   )
]

# calculate MA 
ma = xrtl.WMA(close, timeperiod=60) # you can use also SMA, EMA, etc.
# calcuate ROC
roc = xrtl.ROCP(ma, timeperiod=20)

# We suppose, when abs(roc) < sideways_threshold, the trend is sideways. 
sideways_threshold = 0.03

# positive trend direction
positive_trend = roc > sideways_threshold 
# negtive trend direction
negative_trend = roc < -sideways_threshold 
# sideways
sideways_trend = abs(roc) <= sideways_threshold

# This is a street magic. We will elliminate sideway
# We suppose that a sideways trend after a positive trend is also positive
side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
# and a sideways trend after a negative trend is also negative
side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

# charts with trend indicator

trend_fig = [
    go.Scatter(
        x = pd_time,
        y = ma,
        name='ma',
        line = dict(width=1,color='orange')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(side_positive_trend),
        name='side-positive-trend',
        line = dict(width=1,color='green')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(side_negative_trend),
        name='side-negative-trend',
        line = dict(width=1,color='red')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(positive_trend),
        name='positive-trend',
        line = dict(width=3,color='green')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(negative_trend),
        name='negative-trend',
        line = dict(width=3,color='red')
    ) 
]


# define signals
buy_signal = positive_trend
buy_stop_signal = xruf.logical_or(xruf.logical_not(is_liquid), side_negative_trend)

sell_signal = negative_trend
sell_stop_signal = xruf.logical_or(xruf.logical_not(is_liquid), side_positive_trend)

# calc positions 
position = close.copy(True)
position[:] = np.nan
position = xr.where(buy_signal, 1, position)
position = xr.where(sell_signal, -1, position)
position = xr.where(xruf.logical_and(buy_stop_signal, position.ffill('time') > 0), 0, position)
position = xr.where(xruf.logical_and(sell_stop_signal, position.ffill('time') < 0), 0, position)

position = position.ffill('time').fillna(0)

# calc real orders
real_buy = xruf.logical_and(position > 0, position.shift(time=1) <= 0)
real_sell = xruf.logical_and(position < 0, position.shift(time=1) >= 0)
real_stop = xruf.logical_and(position == 0, position.shift(time=1) != 0)

# plot orders chart
signals_fig=[
    go.Scatter(
        x=close.loc[real_buy].time.to_pandas(),
        y=close.loc[real_buy],
        mode="markers",
        hovertext='buy',
        name="buy",
        marker_size=9,
        marker_color='green'
    ),
    go.Scatter(
        x=close.loc[real_sell].time.to_pandas(),
        y=close.loc[real_sell],
        mode="markers",
        hovertext='sell',
        name="sell",
        marker_size=9,
        marker_color='red'
    ),
    go.Scatter(
        x=close.loc[real_stop].time.to_pandas(),
        y=close.loc[real_stop],
        mode="markers",
        hovertext='stop',
        name="stop",
        marker_size=9,
        marker_color='gray'
    ),
]

# draw chart
fig = go.Figure(data = price_fig + trend_fig + signals_fig)
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

# calc stats
position_with_asset = xr.concat([position], pd.Index([stock_name], name='asset'))
stats = qnstats.calc_stat(data, position_with_asset)
display(stats.to_pandas().tail())

performance = stats.loc[:,"equity"]

# draw performance chart
fig = go.Figure(data = [
    go.Scatter(
        x=performance.time.to_pandas(),
        y=performance,
        hovertext='performance',
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,2.458349,0.017698,0.329764,-0.216941,-0.46795,0.700523,0.231007,1.0,1.0,0.027130,207.5
2020-08-14,2.456158,-0.000891,0.329667,-0.217639,-0.46795,0.681079,0.224529,1.0,1.0,0.027133,207.5
2020-08-17,2.449746,-0.002611,0.329624,-0.219682,-0.46795,0.664486,0.219030,1.0,1.0,0.027147,207.5
2020-08-18,2.470159,0.008333,0.329638,-0.213179,-0.46795,0.679686,0.224050,1.0,1.0,0.027162,207.5
2020-08-19,2.473258,0.001255,0.329431,-0.212192,-0.46795,0.705764,0.232500,1.0,1.0,0.027165,179.0


Now, implement the strategy on multiple assets.

In [6]:
close = data.sel(field='close')
is_liquid = data.sel(field='is_liquid') > 0

# trend
ma = xrtl.WMA(close, timeperiod=100)
roc = xrtl.ROCP(ma, timeperiod=20)

sideways_threshold = 0.02

positive_trend = roc > sideways_threshold 
negative_trend = roc < -sideways_threshold 
sideways_trend = abs(roc) <= sideways_threshold 

side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

# signals
buy_signal = positive_trend
buy_stop_signal = xruf.logical_or(xruf.logical_not(is_liquid), side_negative_trend)

sell_signal = negative_trend
sell_stop_signal = xruf.logical_or(xruf.logical_not(is_liquid), side_positive_trend)

# calc positions 
position = close.copy(True)
position[:] = np.nan

# align signals
buy_signal = xr.align(buy_signal, position, join='right')[0] 
buy_stop_signal = xr.align(buy_stop_signal, position, join='right')[0]
sell_signal = xr.align(sell_signal, position, join='right')[0]
sell_stop_signal = xr.align(sell_stop_signal, position, join='right')[0]

# apply signals to position
position = xr.where(buy_signal, 1, position)
position = xr.where(sell_signal, -1, position)

fp = position.ffill('time')
position = xr.where(xruf.logical_and(buy_stop_signal, fp > 0), 0, position)
position = xr.where(xruf.logical_and(sell_stop_signal, fp < 0), 0, position)

position = position.ffill('time').fillna(0)


# position normalization
output = position/abs(position).sum('asset')


#calc and print stats
stats = qnstats.calc_stat(data, output)
display(output.to_pandas().tail())
display(stats.to_pandas().tail())

asset,AMEX:APT,AMEX:IBIO,AMEX:IGC,AMEX:LNG,NASDAQ:AAL,NASDAQ:AAOI,NASDAQ:AAPL,NASDAQ:AAXN,NASDAQ:ABMD,NASDAQ:ABUS,...,NYSE:XOM,NYSE:XPO,NYSE:XYL,NYSE:YELP,NYSE:YUM,NYSE:YUMC,NYSE:ZAYO,NYSE:ZBH,NYSE:ZEN,NYSE:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2020-08-13,0.0,0.002123,0.0,0.0,-0.002123,0.0,0.002123,0.0,0.002123,0.002123,...,-0.002123,0.002123,0.0,0.0,0.002123,0.0,0.0,0.002123,0.002123,0.002123
2020-08-14,0.0,0.002119,0.0,0.0,-0.002119,0.0,0.002119,0.0,0.002119,0.002119,...,-0.002119,0.002119,0.0,0.0,0.002119,0.0,0.0,0.002119,0.002119,0.002119
2020-08-17,0.0,0.002114,0.0,0.0,-0.002114,0.0,0.002114,0.0,0.002114,0.002114,...,-0.002114,0.002114,0.0,0.0,0.002114,0.0,0.0,0.002114,0.002114,0.002114
2020-08-18,0.0,0.002114,0.0,0.0,-0.002114,0.0,0.002114,0.0,0.002114,0.002114,...,-0.002114,0.002114,0.0,0.0,0.002114,0.0,0.0,0.002114,0.002114,0.002114
2020-08-19,0.0,0.002123,0.0,0.0,-0.002123,0.0,0.002123,0.0,0.002123,0.002123,...,-0.002123,0.002123,0.0,0.0,0.002123,0.0,0.0,0.002123,0.002123,0.002123


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,0.689727,0.004900,0.136397,-0.397575,-0.461502,-0.935567,-0.127608,0.668790,910.0,0.039677,85.030997
2020-08-14,0.688405,-0.001917,0.136314,-0.398729,-0.461502,-0.956891,-0.130437,0.669492,910.0,0.039676,85.018546
2020-08-17,0.693968,0.008082,0.136396,-0.393870,-0.461502,-0.943617,-0.128705,0.674419,910.0,0.039666,85.026651
2020-08-18,0.693820,-0.000213,0.136386,-0.393999,-0.461502,-0.949189,-0.129456,0.678647,910.0,0.039696,85.025208
2020-08-19,0.691822,-0.002880,0.136325,-0.395744,-0.461502,-0.938786,-0.127980,0.677282,910.0,0.039687,82.477923


## Improvement #1

As you can see, the result is not good. This strategy does not work for all assets all time. 
Well, let's try to find the top of assets with good "sharpe_ratio".

In [7]:
top_period = 60
top_size = 60

# normalize output per asset
output_per_asset = output/abs(output)

# calculate stats per asset
stats_per_asset = qnstats.calc_stat(data, output_per_asset, per_asset=True, max_periods=top_period)

# calculate ranks of assets by "sharpe_ratio"
ranks = (-stats_per_asset.sel(field='sharpe_ratio')).rank('asset')
# Select top assets by rank which assets have 'top_period' days ago.
# We assume, that the "sharpe_ratio" of these assets will be good in the next 'top_period' days
rank = ranks.isel(time = -top_period)
top = rank.where(rank <= top_size).dropna('asset').asset

# select top stats
top_stats = stats_per_asset.sel(asset = top.values)

# print results
print("SR tail of the top assets:")
display(top_stats.sel(field='sharpe_ratio').to_pandas().tail())

print("avg SR = ", top_stats[-top_period:].sel(field = 'sharpe_ratio').mean('asset')[-1].item())

SR tail of the top assets:


asset,NASDAQ:ABMD,NASDAQ:AKAM,NASDAQ:AMD,NASDAQ:AMZN,NASDAQ:ATVI,NASDAQ:BBBY,NASDAQ:BMRN,NASDAQ:CTXS,NASDAQ:DOCU,NASDAQ:DXCM,...,NYSE:MOS,NYSE:NEM,NYSE:QGEN,NYSE:SHAK,NYSE:TDOC,NYSE:TPR,NYSE:UAA,NYSE:WFC,NYSE:WORK,NYSE:WWE
time,,,,,,,,,,,,,,,,,,,,,
2020-08-13,4.021411,2.168346,6.299089,5.660006,1.825767,1.025186,4.477076,0.045390,10.198784,0.774553,...,NaN,-0.446625,0.0,-2.001852,0.665999,NaN,0.0,-0.898067,-0.323524,NaN
2020-08-14,3.019749,1.215692,5.573559,4.857192,1.393322,1.033819,4.772357,-0.057422,9.680763,0.627184,...,NaN,-0.253675,0.0,-1.429774,0.676724,NaN,0.0,-0.860617,-0.597809,NaN
2020-08-17,3.263232,1.127862,7.101155,6.029455,2.181114,1.502882,4.855083,0.135342,10.043206,1.092019,...,NaN,0.961933,0.0,-1.630622,1.335898,NaN,0.0,-0.731477,-0.660937,NaN
2020-08-18,2.367485,1.404751,6.438608,7.692369,2.300570,0.888040,3.799753,0.300294,10.418890,0.796289,...,NaN,0.921761,0.0,-1.627080,2.137738,NaN,0.0,-0.707007,-0.643434,NaN
2020-08-19,2.051375,1.570127,7.555300,7.230920,3.136252,0.888040,-0.914205,0.148673,11.874686,1.523054,...,NaN,1.558725,0.0,-1.098794,3.080116,NaN,0.0,-0.278931,-0.717955,NaN


avg SR =  3.457931413334373


The results is good. Now, let's optimize entire output, using this idea:

In [8]:
top_period = 60
top_size = 60
top_step = 60

output_per_asset = output/abs(output)
stats_per_asset = qnstats.calc_stat(data, output_per_asset, per_asset=True, max_periods=top_period)
ranks = (-stats_per_asset.sel(field='sharpe_ratio')).rank('asset')

top_output = output.copy(True)
top_output[:] = 0

for offset in range(top_period - 1, len(ranks), top_step):
    start_date = ranks.time[offset].values
    end_date = ranks.time[min(offset + top_step - 1, len(ranks.time) - 1)].values
    rank = ranks.loc[start_date]
    top = rank.where(rank <= top_size).dropna('asset').asset
    top_output.loc[start_date:end_date, top] = output.loc[start_date:end_date, top]
    
# normalization
top_output = top_output / abs(top_output).sum('asset')
    
#calc stat
top_stats = qnstats.calc_stat(data, top_output)

# display stat
display(top_stats.to_pandas().tail())


# draw performance chart
performance = top_stats.loc[:,"equity"]
fig = go.Figure(data = [
    go.Scatter(
        x=performance.time.to_pandas(),
        y=performance,
        hovertext='performance',
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,0.604007,0.014130,0.223252,-0.519653,-0.599937,-0.744691,-0.166254,0.914894,490.0,0.065833,43.895843
2020-08-14,0.603248,-0.001257,0.223132,-0.520256,-0.599937,-0.761581,-0.169933,0.914894,490.0,0.065849,43.895843
2020-08-17,0.617192,0.023114,0.223539,-0.509167,-0.599937,-0.736242,-0.164579,0.914894,490.0,0.065874,43.895843
2020-08-18,0.619709,0.004079,0.223507,-0.507165,-0.599937,-0.740492,-0.165505,0.914894,490.0,0.065896,43.895843
2020-08-19,0.612294,-0.011966,0.223443,-0.513063,-0.599937,-0.736505,-0.164567,0.914894,490.0,0.065905,43.397115


## Improvement #2

Let's analyze performance trend...

In [9]:
performance = top_stats.loc[:,"equity"]

# performance MA
p_ma = xrtl.WMA(performance, 80)

# ROC
p_roc = xrtl.ROCP(p_ma, timeperiod=20)

# We suppose, when abs(roc) < sideways_threshold, the trend is sideways. 
p_sideways_threshold = 0.01

# positive trend direction
p_positive_trend = p_roc > p_sideways_threshold 
# negtive trend direction
p_negative_trend = p_roc < -p_sideways_threshold 
# sideways
p_sideways_trend = abs(p_roc) <= p_sideways_threshold

# This is a street magic. We will elliminate sideway
# We suppose that a sideways trend after a positive trend is also positive
p_side_positive_trend = p_positive_trend.where(p_sideways_trend == False).ffill('time').fillna(False)
# and a sideways trend after a negative trend is also negative
p_side_negative_trend = p_negative_trend.where(p_sideways_trend == False).ffill('time').fillna(False)

# charts with trend indicator

# draw performance chart
fig = go.Figure(data = [
    go.Scatter(
        x=performance.time.to_pandas(),
        y=performance,
        hovertext='performance',
    ),
    go.Scatter(
        x = p_ma.time.to_pandas(),
        y = p_ma,
        name='ma',
        line = dict(width=1,color='orange')
    ),
    go.Scatter(
        x = p_ma.time.to_pandas(),
        y = p_ma.where(p_side_positive_trend),
        name='side-positive-trend',
        line = dict(width=1,color='green')
    ),
    go.Scatter(
        x = p_ma.time.to_pandas(),
        y = p_ma.where(p_side_negative_trend),
        name='side-negative-trend',
        line = dict(width=1,color='red')
    ),
    go.Scatter(
        x = p_ma.time.to_pandas(),
        y = p_ma.where(p_positive_trend),
        name='positive-trend',
        line = dict(width=3,color='green')
    ),
    go.Scatter(
        x = p_ma.time.to_pandas(),
        y = p_ma.where(p_negative_trend),
        name='negative-trend',
        line = dict(width=3,color='red')
    ) 
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

In [10]:
# now tune output according the performance trend
tuned_output = top_output.where(p_side_positive_trend)
tuned_stats = qnstats.calc_stat(data, tuned_output)

# display stat
display(tuned_stats.to_pandas().tail())


# draw performance chart
t_performance = tuned_stats.loc[:,"equity"]
fig = go.Figure(data = [
    go.Scatter(
        x=t_performance.time.to_pandas(),
        y=t_performance,
        hovertext='performance',
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.017115,0.0,0.105102,-0.15397,-0.16449,0.065453,0.006879,0.0,336.0,0.033113,41.254007
2020-08-14,1.017115,0.0,0.104871,-0.15397,-0.16449,0.027236,0.002856,0.0,336.0,0.033099,41.254007
2020-08-17,1.017115,0.0,0.104852,-0.15397,-0.16449,0.015890,0.001666,0.0,336.0,0.033089,41.254007
2020-08-18,1.017115,0.0,0.104764,-0.15397,-0.16449,-0.007651,-0.000802,0.0,336.0,0.033076,41.254007
2020-08-19,1.017115,0.0,0.104390,-0.15397,-0.16449,0.041619,0.004345,0.0,336.0,0.033053,41.254007


## Statistics

In [11]:
# Calculate statistics on a rolling basis.

# Transactions are punished with slippage equal to a given
# fraction of ATR14 (read more about slippage in our full
# Strategy Buy and Hold template). We evaluate submissions
# using 5% of ATR14 for slippage.

# Mean return, volatility and Sharpe ratio are computed on a
# rolling basis using a lookback period of 3 years.

stat = qnstats.calc_stat(data, tuned_output, slippage_factor=0.05)

display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.017115,0.0,0.105102,-0.15397,-0.16449,0.065453,0.006879,0.0,336.0,0.033113,41.254007
2020-08-14,1.017115,0.0,0.104871,-0.15397,-0.16449,0.027236,0.002856,0.0,336.0,0.033099,41.254007
2020-08-17,1.017115,0.0,0.104852,-0.15397,-0.16449,0.015890,0.001666,0.0,336.0,0.033089,41.254007
2020-08-18,1.017115,0.0,0.104764,-0.15397,-0.16449,-0.007651,-0.000802,0.0,336.0,0.033076,41.254007
2020-08-19,1.017115,0.0,0.104390,-0.15397,-0.16449,0.041619,0.004345,0.0,336.0,0.033053,41.254007


In [12]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

print_stat(stat)

Sharpe Ratio         :  0.045
Mean Return [%]      :  0.432
Volatility [%]       :  9.613
Maximum Drawdown [%] :  16.449


In [13]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"].iloc[(252*3):]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [14]:
# show underwater chart:
UWchart = stat.to_pandas()["underwater"].iloc[(252*3):]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max=0)

In [15]:
# show rolling Sharpe ratio on a 3-year basis:
SRchart = stat.to_pandas()["sharpe_ratio"].iloc[(252*3):]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

In [16]:
# show bias chart:
biaschart = stat.to_pandas()["bias"].iloc[(252*3):]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

## Checks

In [17]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output, data)


Ok. This strategy does not correlate with other strategies.


## Write output

In [18]:
# Finally, we write the last mandatory step for submission,
# namely writing output to file:

qndata.write_output(output)

write output: /root/fractions.nc.gz


At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!

For speeding up evaluation, you can consider submitting a copy with only relevant steps and excluding plots and checks. Your code in the final notebook you submit can be as simple as the following: